In [ ]:
from MDSetup import MDSetup

## Mechanical properties

This notebook uses MDSetup to setup simulations to compute mechanical properties using finite differences

In [ ]:
lammps_setup = MDSetup(
    system_setup="input/setup_mechanical.yaml",
    simulation_default="input/defaults.yaml",
    simulation_ensemble="input/ensemble.yaml",
    simulation_sampling="input/sampling_mechanical.yaml",
    submission_command="sbatch",
)

## Setting up equilibration

In this section the inital system is equilibrated at desired temperature and pressure.

1) Setup simulation


In [ ]:
# Define the simulation folder
simulation_folder = "equilibration"

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = ["em", "npt"]

# Define the simulation time per ensemble in nano seconds (for em the number of iterations is provided in the ensemble yaml)
simulation_times = [0, 0.1]

# Define initial data file (for each pressure & temperature state one)
initial_systems = ["input/anhydrite.data"]

# Provide kwargs that should be passed into the input template directly
input_kwargs = {}

# Define number of copies
copies = 2

# Define the starting number for the first ensemble ( 0{off_set}_ensemble )
off_set = 0

lammps_setup.prepare_simulation(
    folder_name=simulation_folder,
    ensembles=ensembles,
    simulation_times=simulation_times,
    initial_systems=initial_systems,
    input_kwargs=input_kwargs,
    copies=copies,
    off_set=off_set,
)

2) Submit jobs to cluster

In [ ]:
# Submit the simulations
lammps_setup.submit_simulation()

## Setting up deformation

In this section the equilibrated system is deformed in each direction and the corresponding pressure tensor is sampled

1) Setup simulation


In [ ]:
# Define deformation directions
deformation_directions = ["xx", "yy", "zz", "xy", "xz", "yz", "undeformed"]

# Define deformation rates
deformation_rates = [-0.02, -0.01, 0.00, 0.01, 0.02]

# Define the ensembles that should be simulated
ensembles = ["em", "nvt"]

# Define the simulation time per ensemble in nano seconds (for em the number of iterations is provided in the ensemble yaml)
simulation_times = [0.0, 0.01]

# Define initial data file (for each pressure & temperature state one)
initial_systems = ["anhydrite/equilibration/temp_298.1_pres_1.0/copy_0/01_npt/npt.data"]

# Define number of copies
copies = 2

# Define the starting number for the first ensemble ( 0{off_set}_ensemble )
off_set = 0

job_files = [[] for _ in lammps_setup.system_setup["temperature"]]

for deformation_direction in deformation_directions:
    for deformation_rate in deformation_rates:
        # Don't simulate the undeformed system in each deformation direction, as this would simulate the undeformed system 6*no_copies
        if (deformation_rate == 0.0 and deformation_direction != "undeformed") or (
            deformation_rate != 0.0 and deformation_direction == "undeformed"
        ):
            continue
        else:
            # Define the simulation folder
            simulation_folder = (
                f"deformation/{deformation_direction}/{deformation_rate}"
            )

            # Provide kwargs that should be passed into the input template directly
            input_kwargs = {
                "deformation": {
                    "direction": deformation_direction,
                    "rate": deformation_rate,
                }
            }

            lammps_setup.prepare_simulation(
                folder_name=simulation_folder,
                ensembles=ensembles,
                simulation_times=simulation_times,
                initial_systems=initial_systems,
                input_kwargs=input_kwargs,
                copies=copies,
                off_set=off_set,
            )

            for j, files in enumerate(lammps_setup.job_files):
                job_files[j].extend(files)

lammps_setup.job_files = job_files

2) Submit jobs to cluster

In [ ]:
# Submit the simulations
lammps_setup.submit_simulation()

## Analysis

### Analyze lattice properties amd density from equilibration

In [ ]:
# Define the analysis folder
analysis_folder = "equilibration"

# Define analysis ensemble
ensemble = "01_npt"

# Properties to extract
properties = ["a", "b", "c", "alpha", "beta", "gamma"]

# Suffix of output file
output_suffix = "lattice"

# Percentage to discard from beginning of the simulation
time_fraction = 0.0

# Extract properties from LAMMPS and analyse them
lammps_setup.analysis_extract_properties(
    analysis_folder=analysis_folder,
    ensemble=ensemble,
    extracted_properties=properties,
    output_suffix=output_suffix,
    time_fraction=time_fraction,
)

# Properties to extract
properties = ["density"]

# Suffix of output file
output_suffix = "density"

# Extract properties from LAMMPS and analyse them
lammps_setup.analysis_extract_properties(
    analysis_folder=analysis_folder,
    ensemble=ensemble,
    extracted_properties=properties,
    output_suffix=output_suffix,
    time_fraction=time_fraction,
)

### Analyze pressure tensor from production

In [ ]:
# Define analysis folder
analysis_folder = "deformation"

# Define analysis ensemble
ensemble = "01_nvt"

# Define deformation rates to analyse
deformation_rates = [-0.02, -0.01, 0.00, 0.01, 0.02]

# Define method to analysis stiffness tensor
method = "VRH"

# Percentage to discard from beginning of the simulation
time_fraction = 0.0

# Define if stress strain plots should be made
visualize_stress_strain = True

lammps_setup.analysis_mechanical_proerties(
    analysis_folder=analysis_folder,
    ensemble=ensemble,
    deformation_rates=deformation_rates,
    method=method,
    time_fraction=time_fraction,
    visualize_stress_strain=visualize_stress_strain,
)